## ns-3 RIT Scenario: Multi-Pattern Runs and Comparative Analysis (Seed-Averaged)

This notebook runs **multiple ns-3 RIT simulation configurations** and performs
**seed-averaged analysis for reliable comparison**.

- **Outer layer**: Application and node-placement patterns
- **Middle layer**: CSMA / MAC configuration patterns
- **Inner layer**: Multiple random seeds for averaging
- **Total runs**: outer × middle × seeds (e.g., 4 × 8 × 5 = 160)

Simulation results are aggregated to compute **mean, standard deviation, and
inter-seed variance** of key metrics such as PDR, latency, and wake-up time.

All simulations are executed in parallel to efficiently handle seed averaging.

## Load Util Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool, cpu_count
from datetime import datetime

from common.config_utils import SimulationConfig
from common.ns3_utils import run_ns3_simulation, run_ns3_simulation_with_result
from common.io_utils import read_log, find_node_dirs, remove_raw_logs
from common.aggregate_utils import (
    aggregate_app_summary,
    aggregate_mac_summary,
    aggregate_phy_summary,
    aggregate_scenario_summary,
)
from common.log_constants import MAC_LOG_FILES, APP_TXLOG, APP_RXLOG

print("Libraries loaded successfully")
print(f"Available CPU cores: {cpu_count()}")

In [ ]:
# ------------------------------------------------------------
# Execution control settings
#
# These options define how multi-run simulations are executed,
# including rerun policy, error handling, log cleanup, and
# verbosity.
# ------------------------------------------------------------
EXECUTION_CONFIG = {
    'force_rerun': False,           # True: force rerun even if results exist
                                    # False: skip if results already exist
    'cleanup_logs': True,           # True: remove raw logs after successful aggregation
                                    # False: keep raw logs
    'continue_on_error': False,      # True: continue other tasks on error
                                    # False: stop execution on first error
    'allow_partial_results': True,   # True: allow partial results
                                    # False: require complete results only
    'save_error_log': True,          # True: save error logs
                                    # False: do not save error logs
    'verbose_progress': False        # True: detailed progress output
                                    # False: minimal output
}

print(f"Execution configuration: {EXECUTION_CONFIG}")

## Scenario Configuraion (↓↓ edit here ↓↓)

In [ ]:
# ------------------------------------------ ------------------
# Scenario configuration
# Define the 3-layer pattern set here:
# - Outer layer: app + placement (+ optional beacon randomization), up to 8 patterns
# - Middle layer: CSMA / PreCS variations, up to 20 patterns (configurable)
# - Inner layer: seed list for averaging
# ------------------------------------------------------------

# --- Outer patterns (up to 8 patterns) ---
outer_patterns = {
    "periodic(center)": {
        "App": "periodic",
        "Placement": "center"
    },
    "periodic(edge)": {
        "App": "periodic",
        "Placement": "edge"
    },
    "random(center)": {
        "App": "random",
        "Placement": "center"
    },
    "random(edge)": {
        "App": "random",
        "Placement": "edge"
    },
    "periodic(center, brandom)": {
        "App": "periodic",
        "Placement": "center",
        "BeaconRandomize": "true"
    },
    "periodic(edge, brandom)": {
        "App": "periodic",
        "Placement": "edge",
        "BeaconRandomize": "true"
    },
    "random(center, brandom)": {
        "App": "random",
        "Placement": "center",
        "BeaconRandomize": "true"
    },
    "random(edge, brandom)": {
        "App": "random",
        "Placement": "edge",
        "BeaconRandomize": "true"
    }
}

# --- Middle patterns (CSMA / PreCS scenario variants) ---
middle_scenario_configs = {
    "nn": {"params": {}},
    "nn(back)": {"params": {"DWD": 2, "BeaconAck": "true"}},

    "np": {"params": {"BeaconPreCs": "true"}},
    "np(back)": {"params": {"DWD": 2, "BeaconPreCs": "true", "BeaconAck": "true"}},

    "pn": {"params": {"DataPreCs": "true"}},
    "pn(back)": {"params": {"DWD": 2, "DataPreCs": "true", "BeaconAck": "true"}},

    "pp": {"params": {"DataPreCs": "true", "BeaconPreCs": "true"}},
    "pp(back)": {"params": {"DWD": 2, "DataPreCs": "true", "BeaconPreCs": "true", "BeaconAck": "true"}},

    "cn": {"params": {"DataCsma": "true"}},
    "cn(back)": {"params": {"DWD": 2, "DataCsma": "true", "BeaconAck": "true"}},

    "nc": {"params": {"BeaconCsma": "true"}},
    "nc(back)": {"params": {"DWD": 2, "BeaconCsma": "true", "BeaconAck": "true"}},

    "pc": {"params": {"DataPreCs": "true", "BeaconCsma": "true"}},
    "pc(back)": {"params": {"DWD": 2, "DataPreCs": "true", "BeaconCsma": "true", "BeaconAck": "true"}},

    "cp": {"params": {"DataCsma": "true", "BeaconPreCs": "true"}},
    "cp(back)": {"params": {"DWD": 2, "DataCsma": "true", "BeaconPreCs": "true", "BeaconAck": "true"}},

    "cc": {"params": {"DataCsma": "true", "BeaconCsma": "true"}},
    "cc(back)": {"params": {"DWD": 2, "DataCsma": "true", "BeaconCsma": "true", "BeaconAck": "true"}},
}

# --- Seed configuration (inner layer) ---
seed_config = {
    "num_seeds": 10,
    "base_seed": 2000,
    "increment": 1
}

seed_values = [
    seed_config["base_seed"] + i * seed_config["increment"]
    for i in range(seed_config["num_seeds"])
]

# --- Base parameters (shared across all scenarios) ---
base_params = {
    "BI": 3000,
    "TWD": 3000,
    "DWD": 5,
    "Nodes": -1,
    "Days": 1,
    "DR": 0,
    "AppPacketSize": 8,
    "Seed": 210,  # will be overwritten per-seed
    "DataCsma": "false",
    "DataPreCs": "false",
    "DataPreCsB": "false",
    "BeaconCsma": "false",
    "BeaconPreCs": "false",
    "BeaconPreCsB": "false",
    "ContinuousTx": "false",
    "BeaconRandomize": "false",
    "CompactRitDataRequest": "true",
    "BeaconAck": "false",
    "Placement": "edge",
    "Density": "middle",
    "App": "random"
}

def merge_params(*param_dicts):
    """Merge multiple parameter dicts (later dicts override earlier ones)."""
    merged = {}
    for params in param_dicts:
        merged.update(params)
    return merged

# --- Build the full 3-layer task list ---
all_tasks = []
scenario_matrix = {}
base_script = "scratch/rit-grid-converge.cc"

for outer_name, outer_params in outer_patterns.items():
    scenario_matrix[outer_name] = {}

    for middle_name, middle_config in middle_scenario_configs.items():
        scenario_matrix[outer_name][middle_name] = {"seed_tasks": [], "results": []}

        for seed_value in seed_values:
            final_params = merge_params(
                base_params,
                middle_config["params"],
                outer_params,
                {"Seed": seed_value}
            )

            task_name = f"{outer_name}_{middle_name}_seed{seed_value}"

            task_info = {
                "task_name": task_name,
                "params": final_params,
                "base_script": base_script,
                "outer_pattern": outer_name,
                "middle_pattern": middle_name,
                "seed_value": seed_value
            }

            all_tasks.append(task_info)
            scenario_matrix[outer_name][middle_name]["seed_tasks"].append(task_info)

# --- Validation ---
if len(outer_patterns) > 8:
    raise ValueError("Outer pattern count must be <= 8")
if len(middle_scenario_configs) > 20:
    raise ValueError("Middle pattern count must be <= 20")
if seed_config["num_seeds"] <= 0:
    raise ValueError("num_seeds must be >= 1")
if len(all_tasks) == 0:
    raise ValueError("At least one task must be configured")

# --- Summary prints ---
print("=== 3-layer pattern summary ===")
print(f"Outer patterns: {len(outer_patterns)}")
print(f"Middle patterns: {len(middle_scenario_configs)}")
print(f"Seeds per scenario: {seed_config['num_seeds']}")
print(f"Total tasks (outer × middle × seeds): {len(all_tasks)}")
print(f"Scenario count (after seed-averaging): {len(outer_patterns) * len(middle_scenario_configs)}")

print("\n=== Seed settings ===")
print(f"Seed values: {seed_values}")
print(f"Base seed: {seed_config['base_seed']}")
print(f"Increment: {seed_config['increment']}")

print("\n=== Outer patterns ===")
for name, params in outer_patterns.items():
    print(f"{name}:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    print()

print("=== Middle patterns ===")
for name, config in middle_scenario_configs.items():
    print(f"{name}:")
    if config["params"]:
        for key, value in config["params"].items():
            print(f"  {key}: {value}")
    else:
        print("  (same as base params)")
    print()

print("=== Base params ===")
for key, value in base_params.items():
    print(f"  {key}: {value}")


In [ ]:
## Existing-results check & error-tolerance utilities

def is_valid_csv(file_path: str) -> bool:
    """
    Check whether a CSV file is valid.

    A CSV is considered valid if:
      - the file exists
      - the file size is non-zero
      - it can be read by pandas
      - it contains at least one row

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        bool: True if the CSV is valid; otherwise False.
    """
    try:
        if not os.path.exists(file_path):
            return False
        if os.path.getsize(file_path) == 0:
            return False

        df = pd.read_csv(file_path)
        return len(df) > 0
    except Exception:
        return False


def check_existing_task_results(task_info: dict, force_rerun: bool = False) -> dict:
    """
    Check whether an existing task result is complete and can be skipped.

    Args:
        task_info (dict): Task metadata.
        force_rerun (bool): If True, do not skip even if results exist.

    Returns:
        dict: {
            'should_skip': bool,            # True if all required files are valid
            'existing_files': list[str],    # list of valid existing files
            'missing_files': list[str],     # list of missing/invalid files
            'file_status': dict,            # per-file detailed status
            'reason': str                   # short reason message
        }
    """
    if force_rerun:
        return {
            'should_skip': False,
            'existing_files': [],
            'missing_files': [
                'app_summary.csv',
                'mac_summary.csv',
                'phy_summary.csv',
                'scenario_summary.csv'
            ],
            'file_status': {},
            'reason': 'force_rerun=True'
        }

    config = SimulationConfig(task_info["params"], task_info["base_script"])

    required_files = {
        'app_summary.csv': config.get_summary_path("app_summary.csv"),
        'mac_summary.csv': config.get_summary_path("mac_summary.csv"),
        'phy_summary.csv': config.get_summary_path("phy_summary.csv"),
        'scenario_summary.csv': config.get_summary_path("scenario_summary.csv")
    }

    file_status = {}

    for name, path in required_files.items():
        status = {
            'path': path,
            'exists': False,
            'readable': False,
            'has_data': False,
            'file_size': 0,
            'row_count': 0,
            'error': None
        }

        try:
            if not os.path.exists(path):
                status['error'] = "file does not exist"
                file_status[name] = status
                continue

            status['exists'] = True
            status['file_size'] = os.path.getsize(path)

            if status['file_size'] == 0:
                status['error'] = "empty file"
                file_status[name] = status
                continue

            try:
                df = pd.read_csv(path)
                status['readable'] = True
                status['row_count'] = len(df)
                status['has_data'] = len(df) > 0
                if not status['has_data']:
                    status['error'] = "no rows"
            except Exception as csv_error:
                status['readable'] = False
                status['error'] = f"CSV read error: {csv_error}"

        except Exception as e:
            status['error'] = f"file access error: {e}"

        file_status[name] = status

    all_required_valid = all(
        st['exists'] and st['readable'] and st['has_data']
        for st in file_status.values()
    )

    existing_files = [
        name for name, st in file_status.items()
        if st['exists'] and st['readable'] and st['has_data']
    ]
    missing_files = [
        name for name, st in file_status.items()
        if not (st['exists'] and st['readable'] and st['has_data'])
    ]

    return {
        'should_skip': all_required_valid,
        'existing_files': existing_files,
        'missing_files': missing_files,
        'file_status': file_status,
        'reason': 'all required summaries exist and are valid' if all_required_valid else 'missing/invalid summary files'
    }


def load_existing_results(task_info: dict) -> dict:
    """
    Load existing results and return them in the same format as a normal task result.

    Args:
        task_info (dict): Task metadata.

    Returns:
        dict: Task-result dict compatible with run_ns3_simulation_with_result outputs.
    """
    config = SimulationConfig(task_info["params"], task_info["base_script"])

    try:
        scenario_path = config.get_summary_path("scenario_summary.csv")
        scenario_df = pd.read_csv(scenario_path)

        scenario_summary = scenario_df.iloc[0].to_dict() if len(scenario_df) > 0 else {}

        return {
            'success': True,
            'task_name': task_info["task_name"],
            'summary': scenario_summary,
            'outer_pattern': task_info["outer_pattern"],
            'middle_pattern': task_info["middle_pattern"],
            'seed_value': task_info["seed_value"],
            'skipped': True,
            'error': None
        }

    except Exception as e:
        return {
            'success': False,
            'task_name': task_info["task_name"],
            'summary': None,
            'outer_pattern': task_info["outer_pattern"],
            'middle_pattern': task_info["middle_pattern"],
            'seed_value': task_info["seed_value"],
            'skipped': False,
            'error': f"failed to load existing results: {e}"
        }

print("Existing-results check utilities loaded.")


In [ ]:
def run_simulation_with_config(outer_pattern, middle_pattern, seed):
    """
    Run a single ns-3 simulation for the given (outer, middle, seed) configuration.
    """
    # Merge parameters (base + middle overrides + outer overrides + seed)
    final_params = merge_params(
        base_params,
        middle_scenario_configs[middle_pattern]["params"],
        outer_patterns[outer_pattern],
        {"Seed": seed}
    )

    # Build SimulationConfig and run
    config = SimulationConfig(final_params, base_script)
    return run_ns3_simulation_with_result(config)


def process_simulation_statistics(outer_pattern, middle_pattern, seed):
    """
    Run post-processing (aggregation) for a single simulation result.
    Generates CSV summaries and returns in-memory summaries.
    """
    import os
    import traceback

    # Merge parameters
    final_params = merge_params(
        base_params,
        middle_scenario_configs[middle_pattern]["params"],
        outer_patterns[outer_pattern],
        {"Seed": seed}
    )

    config = SimulationConfig(final_params, base_script)

    try:
        print(f"Start statistics: {outer_pattern}_{middle_pattern}_seed{seed}")

        base_dir = os.path.join(config.ns3_working_dir, "logs")
        parameter_dir = config.parameter_dir
        log_base_path = os.path.join(base_dir, parameter_dir)

        print(f"  Log directory: {log_base_path}")

        if not os.path.exists(log_base_path):
            print(f"  Error: log directory not found: {log_base_path}")
            return {'success': False, 'error': f"Log directory not found: {log_base_path}"}

        # Detect node directories
        existing_nodes = []
        for item in os.listdir(log_base_path):
            full = os.path.join(log_base_path, item)
            if item.startswith("node-") and os.path.isdir(full):
                try:
                    existing_nodes.append(int(item.split("-")[1]))
                except (ValueError, IndexError):
                    continue

        existing_nodes.sort()
        print(f"  Detected nodes: {existing_nodes}")

        if not existing_nodes:
            print("  Error: no valid node directories found")
            return {'success': False, 'error': "No valid node directories found"}

        APP_RECV_NODE = 0
        APP_SEND_NODES = [n for n in existing_nodes if n != APP_RECV_NODE]
        MAC_NODES = existing_nodes

        # --- App summary ---
        try:
            print("  Aggregating APP...")
            app_summary = aggregate_app_summary(
                APP_SEND_NODES, APP_RECV_NODE, APP_TXLOG, APP_RXLOG, base_dir, parameter_dir
            )
            app_csv_path = config.get_summary_path("app_summary.csv")
            os.makedirs(os.path.dirname(app_csv_path), exist_ok=True)
            app_summary.to_csv(app_csv_path, index=False)
            print("  APP done.")
        except Exception as e:
            print(f"  APP error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"App summary error: {e}"}

        # --- MAC summary ---
        try:
            print("  Aggregating MAC...")
            mac_summary = aggregate_mac_summary(
                MAC_NODES, MAC_LOG_FILES, base_dir, parameter_dir
            )
            mac_csv_path = config.get_summary_path("mac_summary.csv")
            os.makedirs(os.path.dirname(mac_csv_path), exist_ok=True)
            mac_summary.to_csv(mac_csv_path, index=False)
            print("  MAC done.")
        except Exception as e:
            print(f"  MAC error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"MAC summary error: {e}"}

        # --- PHY summary ---
        try:
            print("  Aggregating PHY...")
            phy_summary = aggregate_phy_summary(
                MAC_NODES, base_dir, parameter_dir
            )
            phy_csv_path = config.get_summary_path("phy_summary.csv")
            os.makedirs(os.path.dirname(phy_csv_path), exist_ok=True)
            phy_summary.to_csv(phy_csv_path, index=False)
            print("  PHY done.")
        except Exception as e:
            print(f"  PHY error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"PHY summary error: {e}"}

        # --- Scenario summary ---
        try:
            print("  Aggregating SCENARIO...")
            scenario_summary = aggregate_scenario_summary(app_summary, phy_summary)
            scenario_summary_df = pd.DataFrame([scenario_summary])
            scenario_csv_path = config.get_summary_path("scenario_summary.csv")
            os.makedirs(os.path.dirname(scenario_csv_path), exist_ok=True)
            scenario_summary_df.to_csv(scenario_csv_path, index=False)
            print("  SCENARIO done.")
        except Exception as e:
            print(f"  SCENARIO error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"Scenario summary error: {e}"}

        print(f"Statistics success: {outer_pattern}_{middle_pattern}_seed{seed}")
        return {
            'success': True,
            'app_summary': app_summary,
            'mac_summary': mac_summary,
            'phy_summary': phy_summary,
            'scenario_summary': scenario_summary
        }

    except Exception as e:
        print(f"Unexpected statistics error: {outer_pattern}_{middle_pattern}_seed{seed}")
        print(f"Error: {e}")
        traceback.print_exc()
        return {'success': False, 'error': f"Unexpected statistics error: {e}"}


def run_single_task_robust(task_info):
    """
    Robust single-task runner.

    Features:
      1) Auto-skip if valid existing summaries are found.
      2) Optionally remove raw logs after successful aggregation.
      3) Keep a structured result dict even on failures (no sys.exit).
    """
    import time

    task_name = task_info["task_name"]
    outer_pattern = task_info["outer_pattern"]
    middle_pattern = task_info["middle_pattern"]
    seed_value = task_info["seed_value"]

    # Existing result check (skip if complete)
    existing_check = check_existing_task_results(
        task_info,
        EXECUTION_CONFIG.get("force_rerun", False)
    )
    if existing_check["should_skip"]:
        return load_existing_results(task_info)

    start_time = time.time()
    result = {
        'success': False,
        'task_name': task_name,
        'summary': None,
        'outer_pattern': outer_pattern,
        'middle_pattern': middle_pattern,
        'seed_value': seed_value,
        'skipped': False,
        'partial_success': False,
        'cleanup_status': 'not_attempted',
        'execution_time': 0,
        'error': None,
        'simulation_success': False,
    }

    try:
        if EXECUTION_CONFIG.get("verbose_progress", False):
            print(f"Start: {task_name}")

        # --- Run simulation ---
        try:
            _ = run_simulation_with_config(outer_pattern, middle_pattern, seed_value)
            result['simulation_success'] = True
        except Exception as sim_error:
            # IMPORTANT: do NOT sys.exit() here (would kill the whole pool/job)
            result['simulation_success'] = False
            result['error'] = f"Simulation error: {sim_error}"
            if EXECUTION_CONFIG.get("verbose_progress", False):
                print(f"Simulation failed: {task_name} ({sim_error})")
            return result

        # --- Run statistics ---
        try:
            stats_result = process_simulation_statistics(outer_pattern, middle_pattern, seed_value)
            if stats_result.get('success', False):
                result['success'] = True
                result['summary'] = stats_result['scenario_summary']

                # Cleanup raw logs only after success
                if EXECUTION_CONFIG.get('cleanup_logs', False):
                    try:
                        final_params = merge_params(
                            base_params,
                            middle_scenario_configs[middle_pattern]["params"],
                            outer_patterns[outer_pattern],
                            {"Seed": seed_value}
                        )
                        config = SimulationConfig(final_params, base_script)
                        log_base_dir = os.path.join(config.ns3_working_dir, "logs")

                        remove_raw_logs(log_base_dir, config.parameter_dir)
                        result['cleanup_status'] = 'completed'
                    except Exception as cleanup_error:
                        result['cleanup_status'] = 'error'
                        if EXECUTION_CONFIG.get('verbose_progress', False):
                            print(f"Cleanup error: {task_name} ({cleanup_error})")

                if EXECUTION_CONFIG.get("verbose_progress", False):
                    print(f"Done: {task_name}")

            else:
                result['partial_success'] = True
                result['error'] = f"Statistics error: {stats_result.get('error', 'unknown error')}"
                if EXECUTION_CONFIG.get("verbose_progress", False):
                    print(f"Statistics failed: {task_name} ({result['error']})")

        except Exception as stats_error:
            result['partial_success'] = True
            result['error'] = f"Statistics processing error: {stats_error}"
            if EXECUTION_CONFIG.get("verbose_progress", False):
                print(f"Statistics exception: {task_name} ({stats_error})")

    except Exception as unexpected_error:
        result['error'] = f"Unexpected error: {unexpected_error}"
        if EXECUTION_CONFIG.get("verbose_progress", False):
            print(f"Unexpected error: {task_name} ({unexpected_error})")

    finally:
        result['execution_time'] = time.time() - start_time

    return result


def generate_execution_report(execution_results):
    """
    Generate a detailed execution report from task results.
    """
    if not execution_results:
        print("No execution results.")
        return {}

    total_tasks = len(execution_results)

    success_count = len([r for r in execution_results if r.get('success', False)])
    skipped_count = len([r for r in execution_results if r.get('skipped', False)])
    failed_count = total_tasks - success_count - skipped_count
    partial_count = len([r for r in execution_results if r.get('partial_success', False)])

    execution_times = [
        r.get('execution_time', 0)
        for r in execution_results
        if not r.get('skipped', False)
    ]

    cleanup_completed = len([r for r in execution_results if r.get('cleanup_status') == 'completed'])
    cleanup_failed = len([r for r in execution_results if r.get('cleanup_status') == 'error'])

    print("\n=== Execution Report ===")
    print(f"Total tasks: {total_tasks}")
    print(f"Success: {success_count} ({success_count/total_tasks*100:.1f}%)")
    print(f"Skipped: {skipped_count} ({skipped_count/total_tasks*100:.1f}%)")
    print(f"Failed: {failed_count} ({failed_count/total_tasks*100:.1f}%)")
    print(f"Partial success: {partial_count} ({partial_count/total_tasks*100:.1f}%)")

    if execution_times:
        print("\nExecution time stats:")
        print(f"  Total: {sum(execution_times):.1f}s")
        print(f"  Mean: {np.mean(execution_times):.1f}s")
        print(f"  Max: {max(execution_times):.1f}s")
        print(f"  Min: {min(execution_times):.1f}s")

    if EXECUTION_CONFIG.get('cleanup_logs', False):
        print("\nRaw-log cleanup stats:")
        print(f"  Completed: {cleanup_completed}")
        print(f"  Failed: {cleanup_failed}")

    return {
        'total_tasks': total_tasks,
        'success_count': success_count,
        'skipped_count': skipped_count,
        'failed_count': failed_count,
        'partial_count': partial_count,
        'execution_times': execution_times,
        'cleanup_completed': cleanup_completed,
        'cleanup_failed': cleanup_failed
    }


## Run parallell simulation

In [ ]:
# Parallel execution settings (fault-tolerant / skip support / raw log cleanup)
max_workers = min(len(all_tasks), cpu_count() - 1)  # Use CPU-1 for stable operation
if max_workers <= 0:
    max_workers = 1

print("=== Parallel Execution Settings ===")
print(f"Total tasks: {len(all_tasks)}")
print(f"Available CPUs: {cpu_count()}")
print(f"Worker processes: {max_workers}")
print("Execution control options:")
for key, value in EXECUTION_CONFIG.items():
    print(f"  {key}: {value}")
print(f"Start time: {datetime.now()}")

# Fault-tolerant parallel execution with skip support
try:
    with Pool(max_workers) as pool:
        task_results = pool.map(run_single_task_robust, all_tasks)

    print(f"\nAll tasks completed: {datetime.now()}")

    # Generate an execution summary report
    generate_execution_report(task_results)

except Exception as e:
    print(f"\nAn error occurred during parallel execution: {str(e)}")
    if not EXECUTION_CONFIG["continue_on_error"]:
        raise

In [ ]:
def run_simulation_with_config(outer_pattern, middle_pattern, seed):
    """
    Run a single ns-3 simulation for the given (outer, middle, seed) configuration.
    """
    # Merge parameters (base + middle overrides + outer overrides + seed)
    final_params = merge_params(
        base_params,
        middle_scenario_configs[middle_pattern]["params"],
        outer_patterns[outer_pattern],
        {"Seed": seed}
    )

    # Build SimulationConfig and run
    config = SimulationConfig(final_params, base_script)
    return run_ns3_simulation_with_result(config)


def process_simulation_statistics(outer_pattern, middle_pattern, seed):
    """
    Run post-processing (aggregation) for a single simulation result.
    Generates CSV summaries and returns in-memory summaries.
    """
    import os
    import traceback

    # Merge parameters
    final_params = merge_params(
        base_params,
        middle_scenario_configs[middle_pattern]["params"],
        outer_patterns[outer_pattern],
        {"Seed": seed}
    )

    config = SimulationConfig(final_params, base_script)

    try:
        print(f"Start statistics: {outer_pattern}_{middle_pattern}_seed{seed}")

        base_dir = os.path.join(config.ns3_working_dir, "logs")
        parameter_dir = config.parameter_dir
        log_base_path = os.path.join(base_dir, parameter_dir)

        print(f"  Log directory: {log_base_path}")

        if not os.path.exists(log_base_path):
            print(f"  Error: log directory not found: {log_base_path}")
            return {'success': False, 'error': f"Log directory not found: {log_base_path}"}

        # Detect node directories
        existing_nodes = []
        for item in os.listdir(log_base_path):
            full = os.path.join(log_base_path, item)
            if item.startswith("node-") and os.path.isdir(full):
                try:
                    existing_nodes.append(int(item.split("-")[1]))
                except (ValueError, IndexError):
                    continue

        existing_nodes.sort()
        print(f"  Detected nodes: {existing_nodes}")

        if not existing_nodes:
            print("  Error: no valid node directories found")
            return {'success': False, 'error': "No valid node directories found"}

        APP_RECV_NODE = 0
        APP_SEND_NODES = [n for n in existing_nodes if n != APP_RECV_NODE]
        MAC_NODES = existing_nodes

        # --- App summary ---
        try:
            print("  Aggregating APP...")
            app_summary = aggregate_app_summary(
                APP_SEND_NODES, APP_RECV_NODE, APP_TXLOG, APP_RXLOG, base_dir, parameter_dir
            )
            app_csv_path = config.get_summary_path("app_summary.csv")
            os.makedirs(os.path.dirname(app_csv_path), exist_ok=True)
            app_summary.to_csv(app_csv_path, index=False)
            print("  APP done.")
        except Exception as e:
            print(f"  APP error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"App summary error: {e}"}

        # --- MAC summary ---
        try:
            print("  Aggregating MAC...")
            mac_summary = aggregate_mac_summary(
                MAC_NODES, MAC_LOG_FILES, base_dir, parameter_dir
            )
            mac_csv_path = config.get_summary_path("mac_summary.csv")
            os.makedirs(os.path.dirname(mac_csv_path), exist_ok=True)
            mac_summary.to_csv(mac_csv_path, index=False)
            print("  MAC done.")
        except Exception as e:
            print(f"  MAC error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"MAC summary error: {e}"}

        # --- PHY summary ---
        try:
            print("  Aggregating PHY...")
            phy_summary = aggregate_phy_summary(
                MAC_NODES, base_dir, parameter_dir
            )
            phy_csv_path = config.get_summary_path("phy_summary.csv")
            os.makedirs(os.path.dirname(phy_csv_path), exist_ok=True)
            phy_summary.to_csv(phy_csv_path, index=False)
            print("  PHY done.")
        except Exception as e:
            print(f"  PHY error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"PHY summary error: {e}"}

        # --- Scenario summary ---
        try:
            print("  Aggregating SCENARIO...")
            scenario_summary = aggregate_scenario_summary(app_summary, phy_summary)
            scenario_summary_df = pd.DataFrame([scenario_summary])
            scenario_csv_path = config.get_summary_path("scenario_summary.csv")
            os.makedirs(os.path.dirname(scenario_csv_path), exist_ok=True)
            scenario_summary_df.to_csv(scenario_csv_path, index=False)
            print("  SCENARIO done.")
        except Exception as e:
            print(f"  SCENARIO error: {e}")
            traceback.print_exc()
            return {'success': False, 'error': f"Scenario summary error: {e}"}

        print(f"Statistics success: {outer_pattern}_{middle_pattern}_seed{seed}")
        return {
            'success': True,
            'app_summary': app_summary,
            'mac_summary': mac_summary,
            'phy_summary': phy_summary,
            'scenario_summary': scenario_summary
        }

    except Exception as e:
        print(f"Unexpected statistics error: {outer_pattern}_{middle_pattern}_seed{seed}")
        print(f"Error: {e}")
        traceback.print_exc()
        return {'success': False, 'error': f"Unexpected statistics error: {e}"}


def run_single_task_robust(task_info):
    """
    Robust single-task runner.

    Features:
      1) Auto-skip if valid existing summaries are found.
      2) Optionally remove raw logs after successful aggregation.
      3) Keep a structured result dict even on failures (no sys.exit).
    """
    import time

    task_name = task_info["task_name"]
    outer_pattern = task_info["outer_pattern"]
    middle_pattern = task_info["middle_pattern"]
    seed_value = task_info["seed_value"]

    # Existing result check (skip if complete)
    existing_check = check_existing_task_results(
        task_info,
        EXECUTION_CONFIG.get("force_rerun", False)
    )
    if existing_check["should_skip"]:
        return load_existing_results(task_info)

    start_time = time.time()
    result = {
        'success': False,
        'task_name': task_name,
        'summary': None,
        'outer_pattern': outer_pattern,
        'middle_pattern': middle_pattern,
        'seed_value': seed_value,
        'skipped': False,
        'partial_success': False,
        'cleanup_status': 'not_attempted',
        'execution_time': 0,
        'error': None,
        'simulation_success': False,
    }

    try:
        if EXECUTION_CONFIG.get("verbose_progress", False):
            print(f"Start: {task_name}")

        # --- Run simulation ---
        try:
            _ = run_simulation_with_config(outer_pattern, middle_pattern, seed_value)
            result['simulation_success'] = True
        except Exception as sim_error:
            # IMPORTANT: do NOT sys.exit() here (would kill the whole pool/job)
            result['simulation_success'] = False
            result['error'] = f"Simulation error: {sim_error}"
            if EXECUTION_CONFIG.get("verbose_progress", False):
                print(f"Simulation failed: {task_name} ({sim_error})")
            return result

        # --- Run statistics ---
        try:
            stats_result = process_simulation_statistics(outer_pattern, middle_pattern, seed_value)
            if stats_result.get('success', False):
                result['success'] = True
                result['summary'] = stats_result['scenario_summary']

                # Cleanup raw logs only after success
                if EXECUTION_CONFIG.get('cleanup_logs', False):
                    try:
                        final_params = merge_params(
                            base_params,
                            middle_scenario_configs[middle_pattern]["params"],
                            outer_patterns[outer_pattern],
                            {"Seed": seed_value}
                        )
                        config = SimulationConfig(final_params, base_script)
                        log_base_dir = os.path.join(config.ns3_working_dir, "logs")

                        remove_raw_logs(log_base_dir, config.parameter_dir)
                        result['cleanup_status'] = 'completed'
                    except Exception as cleanup_error:
                        result['cleanup_status'] = 'error'
                        if EXECUTION_CONFIG.get('verbose_progress', False):
                            print(f"Cleanup error: {task_name} ({cleanup_error})")

                if EXECUTION_CONFIG.get("verbose_progress", False):
                    print(f"Done: {task_name}")

            else:
                result['partial_success'] = True
                result['error'] = f"Statistics error: {stats_result.get('error', 'unknown error')}"
                if EXECUTION_CONFIG.get("verbose_progress", False):
                    print(f"Statistics failed: {task_name} ({result['error']})")

        except Exception as stats_error:
            result['partial_success'] = True
            result['error'] = f"Statistics processing error: {stats_error}"
            if EXECUTION_CONFIG.get("verbose_progress", False):
                print(f"Statistics exception: {task_name} ({stats_error})")

    except Exception as unexpected_error:
        result['error'] = f"Unexpected error: {unexpected_error}"
        if EXECUTION_CONFIG.get("verbose_progress", False):
            print(f"Unexpected error: {task_name} ({unexpected_error})")

    finally:
        result['execution_time'] = time.time() - start_time

    return result


def generate_execution_report(execution_results):
    """
    Generate a detailed execution report from task results.
    """
    if not execution_results:
        print("No execution results.")
        return {}

    total_tasks = len(execution_results)

    success_count = len([r for r in execution_results if r.get('success', False)])
    skipped_count = len([r for r in execution_results if r.get('skipped', False)])
    failed_count = total_tasks - success_count - skipped_count
    partial_count = len([r for r in execution_results if r.get('partial_success', False)])

    execution_times = [
        r.get('execution_time', 0)
        for r in execution_results
        if not r.get('skipped', False)
    ]

    cleanup_completed = len([r for r in execution_results if r.get('cleanup_status') == 'completed'])
    cleanup_failed = len([r for r in execution_results if r.get('cleanup_status') == 'error'])

    print("\n=== Execution Report ===")
    print(f"Total tasks: {total_tasks}")
    print(f"Success: {success_count} ({success_count/total_tasks*100:.1f}%)")
    print(f"Skipped: {skipped_count} ({skipped_count/total_tasks*100:.1f}%)")
    print(f"Failed: {failed_count} ({failed_count/total_tasks*100:.1f}%)")
    print(f"Partial success: {partial_count} ({partial_count/total_tasks*100:.1f}%)")

    if execution_times:
        print("\nExecution time stats:")
        print(f"  Total: {sum(execution_times):.1f}s")
        print(f"  Mean: {np.mean(execution_times):.1f}s")
        print(f"  Max: {max(execution_times):.1f}s")
        print(f"  Min: {min(execution_times):.1f}s")

    if EXECUTION_CONFIG.get('cleanup_logs', False):
        print("\nRaw-log cleanup stats:")
        print(f"  Completed: {cleanup_completed}")
        print(f"  Failed: {cleanup_failed}")

    return {
        'total_tasks': total_tasks,
        'success_count': success_count,
        'skipped_count': skipped_count,
        'failed_count': failed_count,
        'partial_count': partial_count,
        'execution_times': execution_times,
        'cleanup_completed': cleanup_completed,
        'cleanup_failed': cleanup_failed
    }

## Results

In [ ]:
# Three-layer pattern comparison plots (seed-averaged)
import seaborn as sns

# Visualization depends on the number of outer patterns
num_outer = len(outer_patterns)
plt.rcParams['font.family'] = ['DejaVu Sans', 'Noto Sans CJK JP']

if num_outer == 1:
    # Single outer pattern: compare middle patterns
    print("=== Middle-pattern comparison plots (seed-averaged) ===")

    plt.style.use('default')
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(
        f'Middle-pattern comparison (seed-averaged) ({list(outer_patterns.keys())[0]})',
        fontsize=16,
        fontweight='bold'
    )

    # Color palette
    colors = plt.cm.Set3(np.linspace(0, 1, len(middle_scenario_configs)))

    # Extract results for middle patterns only
    middle_df = comparison_df.copy()

    # Plot 1: PDR mean
    axes[0, 0].bar(middle_df['middle_pattern'], middle_df['pdr_mean'], color=colors)
    axes[0, 0].set_title('PDR mean (seed-averaged)', fontweight='bold')
    axes[0, 0].set_ylabel('PDR')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: PDR standard deviation
    axes[0, 1].bar(middle_df['middle_pattern'], middle_df['pdr_std'], color=colors)
    axes[0, 1].set_title('PDR std (seed-averaged)', fontweight='bold')
    axes[0, 1].set_ylabel('PDR std')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: Delay mean
    axes[0, 2].bar(middle_df['middle_pattern'], middle_df['delay_mean'], color=colors)
    axes[0, 2].set_title('Delay mean (seed-averaged)', fontweight='bold')
    axes[0, 2].set_ylabel('Delay (ms)')
    axes[0, 2].tick_params(axis='x', rotation=45)
    axes[0, 2].grid(True, alpha=0.3)

    # Plot 4: Delay standard deviation
    axes[1, 0].bar(middle_df['middle_pattern'], middle_df['delay_std'], color=colors)
    axes[1, 0].set_title('Delay std (seed-averaged)', fontweight='bold')
    axes[1, 0].set_ylabel('Delay std (ms)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3)

    # Plot 5: Wake ratio mean
    axes[1, 1].bar(middle_df['middle_pattern'], middle_df['wake_ratio_mean'], color=colors)
    axes[1, 1].set_title('Wake ratio mean (seed-averaged)', fontweight='bold')
    axes[1, 1].set_ylabel('Wake ratio')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)

    # Plot 6: Wake ratio standard deviation
    axes[1, 2].bar(middle_df['middle_pattern'], middle_df['wake_ratio_std'], color=colors)
    axes[1, 2].set_title('Wake ratio std (seed-averaged)', fontweight='bold')
    axes[1, 2].set_ylabel('Wake ratio std')
    axes[1, 2].tick_params(axis='x', rotation=45)
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    chart_path = "~/workspace/analysis/middle_pattern_seed_averaged_comparison.png"
    plt.savefig(chart_path, dpi=300, bbox_inches='tight')
    plt.show()

else:
    # Multiple outer patterns: three-layer comparison
    print("=== Three-layer pattern comparison (seed-averaged) ===")

    # 1. Performance comparison by outer pattern
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Performance comparison by outer pattern (seed-averaged)', fontsize=16, fontweight='bold')

    # Heatmap: PDR mean
    pdr_pivot = comparison_df.pivot(index='middle_pattern', columns='outer_pattern', values='pdr_mean')
    sns.heatmap(pdr_pivot, annot=True, fmt='.3f', cmap='YlOrRd', ax=axes[0, 0])
    axes[0, 0].set_title('PDR mean (seed-averaged)')

    # Heatmap: Delay mean
    delay_pivot = comparison_df.pivot(index='middle_pattern', columns='outer_pattern', values='delay_mean')
    sns.heatmap(delay_pivot, annot=True, fmt='.1f', cmap='YlGnBu', ax=axes[0, 1])
    axes[0, 1].set_title('Delay mean (ms) (seed-averaged)')

    # Heatmap: Wake ratio mean
    wake_pivot = comparison_df.pivot(index='middle_pattern', columns='outer_pattern', values='wake_ratio_mean')
    sns.heatmap(wake_pivot, annot=True, fmt='.4f', cmap='YlGn', ax=axes[1, 0], vmin=0.005, vmax=0.01)
    axes[1, 0].set_title('Wake ratio mean (seed-averaged)')

    # Outer-pattern average performance summary
    outer_summary = comparison_df.groupby('outer_pattern').agg({
        'pdr_mean': 'mean',
        'delay_mean': 'mean',
        'wake_ratio_mean': 'mean'
    })

    outer_summary.plot(kind='bar', ax=axes[1, 1])
    axes[1, 1].set_title('Average performance by outer pattern (seed-averaged)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].legend(['PDR', 'Delay (ms)', 'Wake ratio'])

    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    chart_path = "~/workspace/analysis/triple_pattern_seed_averaged_comparison.png"
    plt.savefig(chart_path, dpi=300, bbox_inches='tight')
    plt.show()

    # 2. Outer-pattern impact for each middle pattern
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle('Outer-pattern impact by middle pattern (seed-averaged)', fontsize=16, fontweight='bold')

    middle_names = list(middle_scenario_configs.keys())
    for i, middle_name in enumerate(middle_names):
        if i >= 8:  # Up to 8 patterns
            break

        row = i // 4
        col = i % 4

        middle_data = comparison_df[comparison_df['middle_pattern'] == middle_name]
        if not middle_data.empty:
            axes[row, col].bar(middle_data['outer_pattern'], middle_data['pdr_mean'])
            axes[row, col].set_title(f'{middle_name}')
            axes[row, col].set_ylabel('PDR (seed-averaged)')
            axes[row, col].tick_params(axis='x', rotation=45)
            axes[row, col].grid(True, alpha=0.3)

    # Hide unused subplots
    for i in range(len(middle_names), 8):
        row = i // 4
        col = i % 4
        axes[row, col].set_visible(False)

    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    chart_path2 = "~/workspace/analysis/middle_outer_effect_seed_averaged_analysis.png"
    plt.savefig(chart_path2, dpi=300, bbox_inches='tight')
    plt.show()

print("\nSaved comparison plots:")
if num_outer == 1:
    print(f"  - Middle-pattern comparison: {chart_path}")
else:
    print(f"  - Three-layer comparison: {chart_path}")
    print(f"  - Outer impact per middle pattern: {chart_path2}")